In [56]:
def get_embedding(model, face_pixels):
    import numpy as np
    from sklearn.preprocessing import Normalizer
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

In [82]:
class AI_Retraining:
    def __init__(self,path, rotation_range = 60, horizontal_flip = True):
        from mtcnn import MTCNN
        from sklearn.preprocessing import LabelEncoder
        from keras.preprocessing.image import ImageDataGenerator
        
        self.datagen = ImageDataGenerator(rotation_range= rotation_range, horizontal_flip= horizontal_flip)
        self.face_detector = MTCNN()
        self.encoder = LabelEncoder()
        self.X_image = []
        self.y_label = []
        self.path = path
        self.size = (160,160)
    def load_image(self):
        import cv2
        import glob
        try:
            file_path = self.path + "/*.*"
            for i in glob.glob(file_path):
                img = cv2.imread(i)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                self.X_image.append(img)
                label = i.split("/")[-1]
                label = label[:len(label)-9]
                self.y_label.append(label)
        except:
            print("Invalid Path")
    def extract_face(self):
        import cv2
        X=[]
        y=[]
        try:
            for i in range(len(self.X_image)):
                img = self.X_image[i]
                result = self.face_detector.detect_faces(img)
                k=0
                for k in range(len(result)):
                    if(k >= 1):
                        print("Required 1 person per image")
                    else:
                        x1, y1, width, height = result[k]['box']
                        x1, y1 = abs(x1), abs(y1)
                        x2, y2 = x1+width, y1+height
                        face = img[y1:y2, x1:x2]
                        face = cv2.resize(face,self.size)
                        X.append(face)
                        y.append(self.y_label[i])
            self.X_image = X
            self.y_label = y
        except Exception as e:
            print("code failed in extract_face")
            print(e)
    def face_augmentation(self, n = 7):
        import os
        import numpy as np
        try:
            self.augmentation_path = "Desktop/Kashmir Augmented Image" 
            os.mkdir(self.augmentation_path)
            for i in range(len(self.X_image)):
                img = self.X_image[i]
                img = np.reshape(img,(1,160,160,3))
                j=0
                for batch in self.datagen.flow(img, batch_size=1, save_to_dir= self.augmentation_path, save_prefix = self.y_label[i], save_format = "jpg"):
                    if(j >= n):
                        break
                    j+=1
            return self
        except Exception as e:
            print("Code is failed in face_augmentation")
            print(e)
            
    def load_augmented_image(self):
        import cv2
        import glob
        X = []
        y_label = []
        try:
            for i in glob.glob(self.augmentation_path+"/*.*"):
                img = cv2.imread(i)
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                X.append(img)
                y = i.split("/")[2]
                y = y.split("_")
                y = y[0] + y[1]
                y_label.append(y)
            self.X_image = X
            self.y_label = y_label
        except Exception as e:
            print("Code is failed in load_augmented_image")
            print(e)
            
    def features_to_numbers(self,path):
        from keras.models import load_model
        from keras.utils import np_utils
        
        self.facenet_model = load_model(path)
        X = []
        y = []
        for i in range(len(self.X_image)):
            X.append(get_embedding(self.facenet_model, self.X_image[i]))
        y = self.encoder.fit_transform(self.y_label)
        self.no_classes = len(set(y))
        y = np_utils.to_categorical(y, self.no_classes)
        self.X_image = X
        self.y_label = y
    def training_NN(self, epochs = 10):
        from keras.models import Sequential
        from keras.layers import Dense
        import numpy as np
        self.nn_model = Sequential()
        self.nn_model.add(Dense(self.no_classes*3, input_dim = 128 , activation = "relu"))
        self.nn_model.add(Dense(self.no_classes*2, activation = "relu"))
        self.nn_model.add(Dense(self.no_classes, activation = "softmax"))
        self.nn_model.compile(loss = "categorical_crossentropy", optimizer= "adam", metrics=["accuracy"])
        print(self.nn_model.summary())
        self.X_image = np.array(self.X_image)
        self.y_label = np.array(self.y_label)
        self.nn_model.fit(self.X_image, self.y_label, epochs = epochs)
        self.nn_model.save("Desktop/Kashmir Production/new_kashmir3.model")
    def prediction(path):
        import cv2
        import numpy as np
        from keras.models import load_model

        nn_model = load_model("Desktop/Kashmir Production/new_kashmir3.model")
        X_test = []
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = self.face_detector.detect_faces(img)
        for k in range(len(result)):
            x1, y1, width, height = result[k]['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1+width, y1+height
            face = img[y1:y2, x1:x2]
            face = cv2.resize(face,self.size)
            X_test.append(face)
        X_test = np.array(X_test)
        for i in range(len(X_test)):
            img = X_test[i]
            img = np.reshape(img,(1,160,160,3))
            y_pred = nn_model.predict(img)
            y_pred = y_pred[0]
            confidence = max(y_pred)
            y_pred = np.argmax(y_pred)
            y_pred = self.encoder.inverse_transform(y_pred)
            print(y_pred, end=" ")
            print(confidence*100)
            print()

In [83]:
def retraining():
    a = AI_model("Desktop/kashmir_raw_dataset 2")
    a.load_image()
    a.extract_face()
    a.face_augmentation(3)
    a.load_augmented_image()
    a.features_to_numbers("Desktop/facenet_keras.h5/model/facenet_keras.h5")
    a.training_NN(10)

In [ ]:
main()

In [37]:
import os

In [39]:
class AI_pred:
    def __init__(self, path):
        self.path = path
    def prediction():
        import cv2
        import numpy as np
        from keras.models import load_model
        
        img = cv2.imread(path)
        img = cv2.resize(img,(160,160,3))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.array(img)
        img = np.reshape(img, (1,160,160,3))
        

PermissionError: [Errno 1] Operation not permitted: 'Desktop/Kashmir Augmented Image'